In [31]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\Spark\\spark-3.0.1-bin-hadoop2.7'

In [32]:
import csv
lines = open('AnimeData/reviews.csv',encoding='utf-8')
fcsv = csv.reader(lines)
result = list(fcsv)
rates = []
for i in result[1::]:  #remove header row 0
    rates.append((int(i[0]),int(i[2]),float(i[4])))

In [33]:
rates

[(255938, 34096, 8.0),
 (259117, 34599, 10.0),
 (253664, 28891, 7.0),
 (8254, 2904, 9.0),
 (291149, 4181, 10.0),
 (10046, 2904, 10.0),
 (247454, 16664, 6.0),
 (140903, 2904, 8.0),
 (23791, 2904, 10.0),
 (25115, 4181, 4.0),
 (116775, 4672, 8.0),
 (280224, 4181, 9.0),
 (181043, 4181, 10.0),
 (259091, 34599, 8.0),
 (259195, 34599, 9.0),
 (210404, 5114, 10.0),
 (262977, 30276, 8.0),
 (262994, 30276, 10.0),
 (214997, 30276, 7.0),
 (173551, 4181, 10.0),
 (249034, 4181, 5.0),
 (277693, 4181, 10.0),
 (42799, 8038, 10.0),
 (86265, 4181, 9.0),
 (19642, 4181, 10.0),
 (89373, 4181, 10.0),
 (177543, 4181, 9.0),
 (87849, 4181, 9.0),
 (88705, 4181, 5.0),
 (19654, 4181, 10.0),
 (307212, 1253, 8.0),
 (303667, 1425, 7.0),
 (147919, 1425, 7.0),
 (189368, 1425, 6.0),
 (125071, 1425, 6.0),
 (96877, 7588, 7.0),
 (136093, 7588, 7.0),
 (29323, 7588, 7.0),
 (291833, 7588, 9.0),
 (256076, 7588, 10.0),
 (203476, 7588, 10.0),
 (322434, 7588, 10.0),
 (61381, 7588, 7.0),
 (198178, 7588, 9.0),
 (280474, 7588, 9.0),


In [34]:
from pyspark.sql import SparkSession,functions,Row

spark = SparkSession.builder.appName('Anime Recomendation').getOrCreate()
data = spark.createDataFrame(rates,['UID', 'AnimeID', 'Score'])

In [35]:
data.show(10)

+------+-------+-----+
|   UID|AnimeID|Score|
+------+-------+-----+
|255938|  34096|  8.0|
|259117|  34599| 10.0|
|253664|  28891|  7.0|
|  8254|   2904|  9.0|
|291149|   4181| 10.0|
| 10046|   2904| 10.0|
|247454|  16664|  6.0|
|140903|   2904|  8.0|
| 23791|   2904| 10.0|
| 25115|   4181|  4.0|
+------+-------+-----+
only showing top 10 rows



In [36]:
#remove deplicate value of same uid rates same animeID many times
ratings = data.groupBy('UID','AnimeID').avg('score')
ratings.show(10)

+------+-------+----------+
|   UID|AnimeID|avg(score)|
+------+-------+----------+
|287614|   5680|       7.0|
| 28120|   5680|      10.0|
| 94151|     97|      10.0|
| 31995|   4472|       7.0|
|297731|  38958|      10.0|
|217503|   1571|       8.0|
|275482|  19319|       9.0|
|283922|  35860|       8.0|
|240022|  28999|      10.0|
|212633|  20057|      10.0|
+------+-------+----------+
only showing top 10 rows



In [37]:
from pyspark.ml.recommendation import ALS

als = ALS(rank=10, regParam=0.01, userCol='UID', itemCol='AnimeID', ratingCol='avg(score)')
model = als.fit(ratings)

In [38]:
model

ALSModel: uid=ALS_84d3a49ac7d6, rank=10

In [39]:
mostRate = ratings.groupBy('AnimeID').count().filter('count > 50')
mostRate.show(10)

+-------+-----+
|AnimeID|count|
+-------+-----+
|     26|   79|
|  16742|  186|
|  30276|  414|
|  11771|  149|
|  35120|  314|
|  34383|   56|
|  13659|   53|
|   2581|   73|
|    270|   75|
|   1690|   70|
+-------+-----+
only showing top 10 rows



In [40]:
users = ratings.groupBy('UID').count().filter('count = 1').sort('UID')
users.show(10)

+---+-----+
|UID|count|
+---+-----+
|  1|    1|
|  9|    1|
| 10|    1|
| 11|    1|
| 12|    1|
| 13|    1|
| 14|    1|
| 15|    1|
| 16|    1|
| 17|    1|
+---+-----+
only showing top 10 rows



In [41]:
from pyspark.sql.functions import lit

dataForRecom = mostRate.select('AnimeID').withColumn('UID', lit(34))
dataForRecom.show(10)

+-------+---+
|AnimeID|UID|
+-------+---+
|     26| 34|
|  16742| 34|
|  30276| 34|
|  11771| 34|
|  35120| 34|
|  34383| 34|
|  13659| 34|
|   2581| 34|
|    270| 34|
|   1690| 34|
+-------+---+
only showing top 10 rows



In [42]:
file = open('AnimeData/animes.csv',encoding='utf-8')
names = list(csv.reader(file))
anime_names = {}
for name in names[1::]:
    anime_names[int(name[0])] = name[1]

anime_names

{28891: 'Haikyuu!! Second Season',
 23273: 'Shigatsu wa Kimi no Uso',
 34599: 'Made in Abyss',
 5114: 'Fullmetal Alchemist: Brotherhood',
 31758: 'Kizumonogatari III: Reiketsu-hen',
 37510: 'Mob Psycho 100 II',
 199: 'Sen to Chihiro no Kamikakushi',
 38000: 'Kimetsu no Yaiba',
 35247: 'Owarimonogatari 2nd Season',
 2904: 'Code Geass: Hangyaku no Lelouch R2',
 32935: 'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou',
 34096: 'Gintama.',
 15335: 'Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare',
 918: 'Gintama',
 4181: 'Clannad: After Story',
 15417: "Gintama': Enchousen",
 30276: 'One Punch Man',
 16664: 'Kaguya-hime no Monogatari',
 4672: 'Koukaku Kidoutai 2.0',
 8038: 'Nodame Cantabile: Finale - Mine to Kiyora no Saikai',
 7588: 'Saraiya Goyou',
 1253: 'Saint Seiya: Meiou Hades Meikai-hen',
 20767: 'Noragami OVA',
 1425: 'Lupin III: Part II',
 35363: 'Kobayashi-san Chi no Maid Dragon: Valentine, Soshite Onsen! - Amari Kitai Shinaide Kudasai',
 22673: 'Kuroko no Basket

In [43]:
result = model.transform(dataForRecom)
recomemded = result.sort('prediction',ascending=False)
recomemded.show(10)

+-------+---+----------+
|AnimeID|UID|prediction|
+-------+---+----------+
|  28891| 34|  4.599096|
|  10495| 34| 3.3498428|
|   8426| 34| 3.2341256|
|  16774| 34| 3.1218092|
|   6802| 34| 2.9618676|
|  34984| 34|  2.934247|
|  38329| 34| 2.9225383|
|     45| 34|  2.904904|
|     57| 34|  2.898753|
|  21877| 34| 2.5509806|
+-------+---+----------+
only showing top 10 rows



In [52]:
show = recomemded.take(10)

In [59]:
print('Recomended Anime for userid 34:\n')

for anime in show:
    print(anime_names[anime[0]],round(anime[2],2))

Recomended Anime for userid 34:

Haikyuu!! Second Season 4.6
Yuru Yuri 3.35
Hourou Musuko 3.23
Inferno Cop 3.12
So Ra No Wo To 2.96
Koi wa Ameagari no You ni 2.93
Seishun Buta Yarou wa Yumemiru Shoujo no Yume wo Minai 2.92
Rurouni Kenshin: Meiji Kenkaku Romantan 2.9
Beck 2.9
High Score Girl 2.55
